<a href="https://colab.research.google.com/github/Komal-Zia/PRcode/blob/main/Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow --quiet


In [2]:
# Install CEEMDAN from GitHub
!pip install git+https://github.com/laszukdawid/PyEMD.git --quiet


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.14.4 requires dill<0.3.8,>=0.3.0, but you have dill 0.4.0 which is incompatible.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PyEMD import CEEMDAN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense

# -------------------------------
# Load and preprocess the dataset
# -------------------------------
from google.colab import files
uploaded = files.upload()

# Get the uploaded filename dynamically
filename = list(uploaded.keys())[0]

# Load the dataset
df = pd.read_csv(filename, parse_dates=['Datetime'], index_col='Datetime')
df = df.dropna().sort_index()
print("Columns:", df.columns)


# Use first column (or change if needed)
column_name = df.columns[0]
data_raw = df[column_name].values.reshape(-1, 1)

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data_raw)

# -------------------------------
# CEEMDAN Decomposition
# -------------------------------
def apply_ceemdan(data):
    ceemdan = CEEMDAN()
    ceemdan.trials = 100
    ceemdan.noise_seed(42)
    imfs = ceemdan.ceemdan(data.flatten())
    return imfs

# -------------------------------
# Create Sequences
# -------------------------------
def create_sequences(data, time_steps=24):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps])
        y.append(data[i+time_steps])
    return np.array(X), np.array(y)

# -------------------------------
# CNN-BiLSTM Model
# -------------------------------
def build_model(input_shape, units, filters, lr):
    model = Sequential([
        Conv1D(filters=int(filters), kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Bidirectional(LSTM(int(units))),
        Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='mse')
    return model

# -------------------------------
# Custom ISSA (Simplified)
# -------------------------------
def simple_issa(fitness_fn, n_dim, n_agents=5, lower=None, upper=None, max_iter=5):
    lower = np.array(lower)
    upper = np.array(upper)
    pop = np.random.uniform(lower, upper, (n_agents, n_dim))
    best = pop[0]
    best_score = fitness_fn(best)

    for _ in range(max_iter):
        for i in range(n_agents):
            candidate = pop[i] + np.random.uniform(-1, 1, n_dim) * (best - pop[i])
            candidate = np.clip(candidate, lower, upper)
            score = fitness_fn(candidate)
            if score < best_score:
                best = candidate
                best_score = score
            pop[i] = candidate

    return best

# -------------------------------
# Objective Function
# -------------------------------
def objective_fn(params, X_train, y_train, X_val, y_val):
    units, filters, lr = int(params[0]), int(params[1]), float(params[2])
    model = build_model((X_train.shape[1], X_train.shape[2]), units, filters, lr)
    model.fit(X_train, y_train, epochs=3, batch_size=32, verbose=0)
    preds = model.predict(X_val)
    return mean_squared_error(y_val, preds)

# -------------------------------
# CICB Training Function
# -------------------------------
def train_cicb_model(data, time_steps=24):
    imfs = apply_ceemdan(data)
    prediction_total = np.zeros((len(data) - time_steps, 1))

    for i, imf in enumerate(imfs):
        print(f"Training on IMF {i+1}/{len(imfs)}...")
        X, y = create_sequences(imf.reshape(-1, 1), time_steps)
        split = int(len(X) * 0.8)
        X_train, X_val = X[:split], X[split:]
        y_train, y_val = y[:split], y[split:]

        # Define fitness function for this IMF
        def fitness_fn(params): return objective_fn(params, X_train, y_train, X_val, y_val)

        best_params = simple_issa(
            fitness_fn=fitness_fn,
            n_dim=3,
            n_agents=5,
            lower=[32, 16, 0.0001],
            upper=[128, 64, 0.01],
            max_iter=3
        )

        units, filters, lr = best_params
        model = build_model((X.shape[1], X.shape[2]), int(units), int(filters), lr)
        model.fit(X_train, y_train, epochs=3, batch_size=32, verbose=0)
        pred = model.predict(X_val)

        padded_pred = np.zeros((len(data) - time_steps, 1))
        padded_pred[split:] = pred
        prediction_total += padded_pred

    return prediction_total

# -------------------------------
# Run the Model
# -------------------------------
time_steps = 24
pred_scaled = train_cicb_model(data_scaled, time_steps)
true_scaled = data_scaled[-len(pred_scaled):]

# Inverse transform to original scale
pred = scaler.inverse_transform(pred_scaled)
true = scaler.inverse_transform(true_scaled)

# -------------------------------
# Plot Results
# -------------------------------
plt.figure(figsize=(14, 5))
plt.plot(true, label='True Load')
plt.plot(pred, label='Predicted Load')
plt.title("STLF using CEEMDAN + Custom ISSA + CNN-BiLSTM")
plt.xlabel("Time Steps")
plt.ylabel("Load (MW)")
plt.legend()
plt.grid(True)
plt.show()


Saving PJM_Load_hourly.csv to PJM_Load_hourly (2).csv
Columns: Index(['PJM_Load_MW'], dtype='object')


<ipython-input-6-d9be56b1d220>:21: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv(filename, parse_dates=['Datetime'], index_col='Datetime')


Training on IMF 1/12...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


206/206 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step


...

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


206/206 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


206/206 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
